# WebQore demo

WebQoreの使用方法のデモ。

タスクは心電図の波形データから正常か不整脈かを判断する二値分類。

以下の日本メディカルAI学会のオンライン講義資料を参考にした。

https://japan-medical-ai.github.io/medical-ai-course-materials/notebooks/08_Sequential_Data_Analysis_with_Deep_Learning.html

## ライブラリのインポート

In [1]:
import matplotlib.pyplot as plt
from imblearn.datasets import make_imbalance
from sklearn.metrics import classification_report
import numpy as np

from qore_sdk.utils import sliding_window
from qore_sdk.featurizer import Featurizer
from qore_sdk.client import WebQoreClient

from dataloader import load_dataset

%matplotlib inline

## アカウント情報とハイパーパラメータの設定

In [2]:
# 取得したアカウント情報
username = ''
password = ''
endpoint = ''

width = 144
stepsize = 36
n_filters = 40

## 前処理

In [3]:
# データセットのダウンロード
X_train, X_test, y_train, y_test = load_dataset()
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)

X_train.shape: (47738, 720)
X_test.shape: (45349, 720)


In [4]:
# データ数が多いので減らす。
## 学習データ
_, counts = np.unique(y_train, return_counts=True)
n_samples_per_class = counts.min() // 10
X_train, y_train = make_imbalance(X_train, y_train, {0:n_samples_per_class, 1:n_samples_per_class})
print('X_train.shape:', X_train.shape)
## 検証データ
_, counts = np.unique(y_test, return_counts=True)
X_test, y_test = make_imbalance(X_test, y_test, {0:counts[0]//100, 1:counts[1]//100})
print('X_test.shape:', X_test.shape)

X_train.shape: (748, 720)
X_test.shape: (453, 720)


In [5]:
# Qoreへの入力は(サンプル数,時系列長,変量数)なので、それに合わせるためにデータを分割する。
X_train = sliding_window(X_train, width, stepsize)
X_test = sliding_window(X_test, width, stepsize)
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)

X_train.shape: (748, 17, 144)
X_test.shape: (453, 17, 144)


## 特徴抽出

In [6]:
%%time
featurizer = Featurizer(n_filters)
X_train = featurizer.featurize(X_train, axis=2)
X_test = featurizer.featurize(X_test, axis=2)
print(X_train.shape, X_test.shape)

(748, 17, 40) (453, 17, 40)
CPU times: user 369 ms, sys: 248 ms, total: 617 ms
Wall time: 148 ms


## 学習

In [7]:
%%time
client = WebQoreClient(username, password, endpoint=endpoint)
time_ = client.classifier_train(X_train, y_train)
print('Time:.', time_, 'min.')

Time:. {'res': 'ok', 'train_time': 0.04814064900080363} min.
CPU times: user 624 ms, sys: 240 ms, total: 864 ms
Wall time: 3.53 s


## 推論

In [8]:
%%time
res = client.classifier_predict(X_test)

CPU times: user 282 ms, sys: 27.1 ms, total: 310 ms
Wall time: 1.43 s


In [9]:
report = classification_report(y_test, res['Y'])
print(report)

              precision    recall  f1-score   support

           0       0.99      0.94      0.96       421
           1       0.53      0.94      0.67        32

    accuracy                           0.94       453
   macro avg       0.76      0.94      0.82       453
weighted avg       0.96      0.94      0.94       453

